In [2]:
import pandas as pd
import geopandas as gpd
from haversine import haversine, Unit
import ast
import time

In [3]:
smart_data = pd.read_csv('DETERMINADOS/smart_dos_a_seis_transacciones.csv', sep = ',')
smart_data["FechaHoraValidacion"] = pd.to_datetime(smart_data["FechaHoraValidacion"])

In [40]:
stops_data = pd.read_csv('stop_data_linea.csv', sep = ',')
stops_data['ID-PARADA_2'] = stops_data['ID-PARADA_2'].apply(ast.literal_eval)
stops_instancia = stops_data

In [5]:
# Ordenar los datos por usuario, fecha y hora
smart_data = smart_data.sort_values(['CodigoTarjeta', 'Fecha', 'FechaHoraValidacion'])

# Obtener el primer y último registro del día por usuario
primeros = smart_data.groupby(['CodigoTarjeta', 'Fecha']).first().reset_index()
ultimos = smart_data.groupby(['CodigoTarjeta', 'Fecha']).last().reset_index()

# Añadir una columna para identificar si es el primero o el último
primeros['TipoRegistro'] = 'Primero'
ultimos['TipoRegistro'] = 'Ultimo'

# Combinar ambos en un solo DataFrame
registros_dia = pd.concat([primeros, ultimos], ignore_index=True)

# Orden final
registros_dia = registros_dia.sort_values(['CodigoTarjeta', 'Fecha', 'TipoRegistro'])


In [28]:
registros_dia

,CodigoTarjeta,Fecha,CodigoParada,NombreParada,CodigoLinea,NombreLinea,FechaHoraValidacion,CompaniaT,RegistroMunicipal,TipoTarifa,Categoria,Semana,TipoRegistro
0,CURA0000017701,2023-10-03,1137,ENTRADA A LA CATOLICA,9102,Linea 4,2023-10-03 12:57:00,06 - COMTRANUTOME S.A,267,1,1,40,Primero
2395302,CURA0000017701,2023-10-03,0,No Determinado,6005,15,2023-10-03 13:28:46,06 - COMTRANUTOME S.A,354,1,1,40,Ultimo
1,CURA0000083349,2023-10-14,1702,TERMINAL TERRESTRE,2004,28 - Sidcay,2023-10-14 12:00:51,06 - COMTRANUTOME S.A,306,1,1,41,Primero
2395303,CURA0000083349,2023-10-14,1702,TERMINAL TERRESTRE,2004,28 - Sidcay,2023-10-14 12:00:56,06 - COMTRANUTOME S.A,306,1,1,41,Ultimo
2,CURA0000174182,2023-10-09,1702,TERMINAL TERRESTRE,5001,12,2023-10-09 11:33:35,03 - RICAURTESA S.A.,148,1,1,41,Primero
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4790601,CURS0300000004,2023-10-02,2317,JUAN JARAMILLO Y BENIGNO MALO,1003,20,2023-10-02 17:36:30,01 - LANCOMTRI S.A,28,0,3,40,Ultimo
2395300,CURS0300000004,2023-10-14,794,EL EJECUTIVO,9100,100,2023-10-14 12:02:28,07 - COMCUETU S.A,376,0,3,41,Primero
4790602,CURS0300000004,2023-10-14,2353,JUAN JARAMILLO Y BORRERO,1003,20,2023-10-14 16:53:20,01 - LANCOMTRI S.A,22,0,3,41,Ultimo
2395301,CURS0300000004,2023-10-24,794,EL EJECUTIVO,1003,20,2023-10-24 13:56:18,01 - LANCOMTRI S.A,33,0,3,43,Primero


## Regresos Consecutivos

In [34]:
import pandas as pd

# 1. Ordenar por tarjeta y fecha-hora
smart_data = smart_data.sort_values(['CodigoTarjeta', 'FechaHoraValidacion'])

# 2. Obtener primer y último registro por día y usuario
primeros = (
    smart_data.groupby(['CodigoTarjeta', 'Fecha'])
    .first()[['CodigoParada', 'CodigoLinea', 'FechaHoraValidacion']]
    .reset_index()
)
ultimos = (
    smart_data.groupby(['CodigoTarjeta', 'Fecha'])
    .last()[['CodigoParada', 'CodigoLinea', 'FechaHoraValidacion']]
    .reset_index()
)

# 3. Identificar días consecutivos (último día N -> primero día N+1)
ultimos['Fecha'] = pd.to_datetime(ultimos['Fecha'])
ultimos = ultimos.sort_values(['CodigoTarjeta', 'Fecha'])
ultimos['Fecha_siguiente'] = ultimos.groupby('CodigoTarjeta')['Fecha'].shift(-1)
ultimos['Diferencia_dias'] = (ultimos['Fecha_siguiente'] - ultimos['Fecha']).dt.days

# 4. Filtrar solo consecutivos y unir con primeros registros del día siguiente
consecutivos = ultimos[ultimos['Diferencia_dias'] == 1].merge(
    primeros,
    left_on=['CodigoTarjeta', 'Fecha_siguiente'],
    right_on=['CodigoTarjeta', 'Fecha'],
    suffixes=('_ultimo', '_primero')
)

# 5. Seleccionar solo columnas relevantes
columnas_finales = [
    'CodigoTarjeta',
    'Fecha_ultimo', 'CodigoParada_ultimo', 'CodigoLinea_ultimo', 'FechaHoraValidacion_ultimo',
    'Fecha_primero', 'CodigoParada_primero', 'CodigoLinea_primero', 'FechaHoraValidacion_primero'
]
consecutivos = consecutivos[columnas_finales]


In [39]:
consecutivos

,CodigoTarjeta,Fecha_ultimo,CodigoParada_ultimo,CodigoLinea_ultimo,FechaHoraValidacion_ultimo,Fecha_primero,CodigoParada_primero,CodigoLinea_primero,FechaHoraValidacion_primero
0,CURA0010000028,2023-10-05,1363,7001,2023-10-05 13:23:07,2023-10-06,1363,7001,2023-10-06 13:49:29
1,CURA0010000028,2023-10-10,1914,7001,2023-10-10 18:38:12,2023-10-11,1958,7001,2023-10-11 14:19:39
2,CURA0010000050,2023-10-07,1772,9100,2023-10-07 17:04:32,2023-10-08,1661,9100,2023-10-08 08:47:47
3,CURA0010000050,2023-10-08,1676,1004,2023-10-08 11:24:58,2023-10-09,1721,6007,2023-10-09 10:53:59
4,CURA0010000050,2023-10-09,1772,9100,2023-10-09 11:28:59,2023-10-10,1721,2004,2023-10-10 09:19:55
...,...,...,...,...,...,...,...,...,...
1121214,CURS0030295046,2023-10-30,1794,7001,2023-10-30 21:32:15,2023-10-31,1807,7001,2023-10-31 13:23:47
1121215,CURS0030303555,2023-10-23,2465,6007,2023-10-23 15:02:01,2023-10-24,3077,4002,2023-10-24 16:17:29
1121216,CURS0030303555,2023-10-24,3027,4002,2023-10-24 18:06:01,2023-10-25,4214,2002,2023-10-25 13:12:44
1121217,CURS0030319717,2023-10-15,2484,4004,2023-10-15 10:44:13,2023-10-16,200,4004,2023-10-16 07:04:24


In [41]:
df_pares = consecutivos

In [43]:
inicio = time.time()
resultados_inferencia = []

# Diccionarios para acceso rápido
paradas_2_dict = stops_instancia.set_index('ID-PARADA')['ID-PARADA_2'].to_dict()
lineas_dict = stops_instancia.set_index('ID-PARADA')['ID-LINEA'].to_dict()
coords_dict = stops_instancia.set_index('ID-PARADA')[['LATITUD', 'LONGITUD']].to_dict('index')

# Recorrer cada fila con un par ida-regreso
for _, fila in df_pares.iterrows():
    try:
        usuario = fila['CodigoTarjeta']
        # Ascenso del viaje (IDA)
        parada_ida = fila['CodigoParada_ultimo']
        linea_ida = fila['CodigoLinea_ultimo']
        fecha_hora_ida = fila['FechaHoraValidacion_ultimo']

        # Regreso (usado para inferir el descenso)
        parada_regreso = fila['CodigoParada_primero']
        linea_regreso = fila['CodigoLinea_primero']
        fecha_hora_regreso = fila['FechaHoraValidacion_primero']

        # Buscar paradas cercanas al ascenso del regreso
        paradas_cercanas = paradas_2_dict.get(parada_regreso, [])
        if isinstance(paradas_cercanas, str):
            paradas_cercanas = eval(paradas_cercanas)
        if not paradas_cercanas:
            continue

        # Filtrar paradas con la misma línea del regreso
        paradas_comunes = [
            p for p in paradas_cercanas
            if lineas_dict.get(p) == linea_ida
        ]

        # ⚠️ Si no hay paradas comunes, usar todas las cercanas
        if not paradas_comunes:
            paradas_comunes = paradas_cercanas

        # Coordenadas para calcular distancia
        coord_regreso = coords_dict.get(parada_regreso)
        if not coord_regreso:
            continue

        # Inferir parada más cercana
        parada_descenso = min(
            paradas_comunes,
            key=lambda p: haversine(coord_regreso.values(), coords_dict[p].values(), unit=Unit.METERS)
        )

        resultados_inferencia.append({
            'CodigoTarjeta': usuario,
            'FechaHoraAscenso': fecha_hora_ida,
            'ParadaAscenso_Nombre': parada_ida,
            'LineaViaje': linea_ida,
            'ParadaRegreso': parada_regreso,
            'FechaHoraRegreso': fecha_hora_regreso,
            'ParadaDescenso_Inferida': parada_descenso
        })

    except Exception as e:
        print(f"⚠️ Error con usuario {usuario} en fecha {fecha}: {e}")
        continue

fin = time.time()
print(f"⏱️ Tiempo de ejecución: {fin - inicio:.2f} segundos")


⏱️ Tiempo de ejecución: 121.60 segundos


In [44]:
df_inferencias_final = pd.DataFrame(resultados_inferencia)

In [46]:
df_inferencias_final.to_csv('DETERMINADOS/retornos_consecutivos_inferidos.csv', index = False)

## Regresos No Consecutivos

In [30]:
import pandas as pd

# 1. Ordenar datos por tarjeta y fecha-hora
smart_data = smart_data.sort_values(['CodigoTarjeta', 'FechaHoraValidacion'])

# 2. Obtener primer y último registro por día y usuario (manteniendo 'Fecha' como referencia)
primeros = (
    smart_data.groupby(['CodigoTarjeta', 'Fecha'])
    .first()[['CodigoParada', 'CodigoLinea', 'FechaHoraValidacion']]
    .reset_index()
    .rename(columns={
        'CodigoParada': 'CodigoParada_primero',
        'CodigoLinea': 'CodigoLinea_primero',
        'FechaHoraValidacion': 'FechaHoraValidacion_primero',
        'Fecha': 'Fecha_primero'  # Renombrar 'Fecha' para el primer registro
    })
)

ultimos = (
    smart_data.groupby(['CodigoTarjeta', 'Fecha'])
    .last()[['CodigoParada', 'CodigoLinea', 'FechaHoraValidacion']]
    .reset_index()
    .rename(columns={
        'CodigoParada': 'CodigoParada_ultimo',
        'CodigoLinea': 'CodigoLinea_ultimo',
        'FechaHoraValidacion': 'FechaHoraValidacion_ultimo',
        'Fecha': 'Fecha_ultimo'  # Renombrar 'Fecha' para el último registro
    })
)

# 3. Unir primeros y últimos registros (usando 'CodigoTarjeta' como clave)
registros_dia = pd.merge(
    primeros,
    ultimos,
    left_on=['CodigoTarjeta', 'Fecha_primero'],
    right_on=['CodigoTarjeta', 'Fecha_ultimo'],
    how='outer'
)

# 4. Identificar días consecutivos (último día N vs. primero día N+1)
registros_dia['Fecha_ultimo'] = pd.to_datetime(registros_dia['Fecha_ultimo'])
registros_dia = registros_dia.sort_values(['CodigoTarjeta', 'Fecha_ultimo'])
registros_dia['Fecha_siguiente'] = registros_dia.groupby('CodigoTarjeta')['Fecha_ultimo'].shift(-1)
registros_dia['Diferencia_dias'] = (registros_dia['Fecha_siguiente'] - registros_dia['Fecha_ultimo']).dt.days

# 5. Separar en consecutivos y no consecutivos
consecutivos = registros_dia[registros_dia['Diferencia_dias'] == 1].copy()
no_consecutivos = registros_dia[registros_dia['Diferencia_dias'] != 1].copy()

# 6. Ajustar no_consecutivos (fechas iguales para mismo día)
no_consecutivos['Fecha_primero'] = no_consecutivos['Fecha_ultimo']  # Mismo día

# 7. Seleccionar columnas finales
columnas = [
    'CodigoTarjeta',
    'Fecha_ultimo', 'CodigoParada_ultimo', 'CodigoLinea_ultimo', 'FechaHoraValidacion_ultimo',
    'Fecha_primero', 'CodigoParada_primero', 'CodigoLinea_primero', 'FechaHoraValidacion_primero'
]

no_consecutivos = no_consecutivos[columnas]

In [47]:
df_pares = no_consecutivos

In [50]:
inicio = time.time()
resultados_inferencia = []

# Diccionarios para acceso rápido
paradas_2_dict = stops_instancia.set_index('ID-PARADA')['ID-PARADA_2'].to_dict()
lineas_dict = stops_instancia.set_index('ID-PARADA')['ID-LINEA'].to_dict()
coords_dict = stops_instancia.set_index('ID-PARADA')[['LATITUD', 'LONGITUD']].to_dict('index')

# Recorrer cada fila con un par ida-regreso
for _, fila in df_pares.iterrows():
    try:
        usuario = fila['CodigoTarjeta']
        # Ascenso del viaje (IDA)
        parada_ida = fila['CodigoParada_ultimo']
        linea_ida = fila['CodigoLinea_ultimo']
        fecha_hora_ida = fila['FechaHoraValidacion_ultimo']

        # Regreso (usado para inferir el descenso)
        parada_regreso = fila['CodigoParada_primero']
        linea_regreso = fila['CodigoLinea_primero']
        fecha_hora_regreso = fila['FechaHoraValidacion_primero']

        # Buscar paradas cercanas al ascenso del regreso
        paradas_cercanas = paradas_2_dict.get(parada_regreso, [])
        if isinstance(paradas_cercanas, str):
            paradas_cercanas = eval(paradas_cercanas)
        if not paradas_cercanas:
            continue

        # Filtrar paradas con la misma línea del regreso
        paradas_comunes = [
            p for p in paradas_cercanas
            if lineas_dict.get(p) == linea_ida
        ]

        # ⚠️ Si no hay paradas comunes, usar todas las cercanas
        if not paradas_comunes:
            paradas_comunes = paradas_cercanas

        # Coordenadas para calcular distancia
        coord_regreso = coords_dict.get(parada_regreso)
        if not coord_regreso:
            continue

        # Inferir parada más cercana
        parada_descenso = min(
            paradas_comunes,
            key=lambda p: haversine(coord_regreso.values(), coords_dict[p].values(), unit=Unit.METERS)
        )

        resultados_inferencia.append({
            'CodigoTarjeta': usuario,
            'FechaHoraAscenso': fecha_hora_ida,
            'ParadaAscenso_Nombre': parada_ida,
            'LineaViaje': linea_ida,
            'ParadaRegreso': parada_regreso,
            'FechaHoraRegreso': fecha_hora_regreso,
            'ParadaDescenso_Inferida': parada_descenso
        })

    except Exception as e:
        print(f"⚠️ Error con usuario {usuario} en fecha {fecha}: {e}")
        continue

fin = time.time()
print(f"⏱️ Tiempo de ejecución: {fin - inicio:.2f} segundos")
df_inferencias_final = pd.DataFrame(resultados_inferencia)

⏱️ Tiempo de ejecución: 139.61 segundos


In [52]:
df_inferencias_final.to_csv('DETERMINADOS/retornos_no_consecutivos_inferidos.csv', index = False)